# Display Phoenix model of Brown Dwarf

In [ ]:
from __future__ import division, print_function 
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline

from Spectrum import Spectrum

In [ ]:
# From plot fits
pathwave = "/home/jneal/Phd/data/phoenixmodels/WAVE_PHOENIX-ACES-AGSS-COND-2011.fits"
model = "/home/jneal/Phd/data/phoenixmodels/HD30501b-lte02500-5.00-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
star_model = "/home/jneal/Phd/data/phoenixmodels/HD30501-lte05200-4.50-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"

I_mod = fits.getdata(model)
I_star = fits.getdata(star_model)
hdr = fits.getheader(star_model)
hdr_star = fits.getheader(star_model)
w_mod = fits.getdata(pathwave)

w_mod /= 10   # turn into nm

w0 = 2112
w1 = 2124

# Specifiying wavelength range
i = (w_mod > w0) & (w_mod < w1)
w_mod = w_mod[i]
I_mod = I_mod[i]
I_star = I_star[i]
        
# https://phoenix.ens-lyon.fr/Grids/FORMAT
# I_mod = 10 ** (I_mod-8.0)
I_mod /= np.median(I_mod)
# Normalization (use first 50 points below 1.2 as continuum)
mod_maxes = I_mod[(I_mod < 1.2)].argsort()[-50:][::-1]
I_mod /= np.median(I_mod[mod_maxes])

#I_mod, w_mod = dopplerShift(wvl=w_mod, flux=I_mod, v=rv1, fill_value=0.95)
I_star /= np.median(I_star)
# Normalization (use first 50 points below 1.2 as continuum)
maxes = I_star[(I_star < 1.2)].argsort()[-50:][::-1]
I_star /= np.median(I_star[maxes])    
    
#spec_mod = Spectrum.Spectrum(I_mod, w_mod, calibrated=True)    
#spec_star = Spectrum.Spectrum(I_star, w_mod, calibrated=True)  

In [ ]:
plt.plot(w_mod, I_mod , label="2500 K")
#plt.plot(w_mod, (I_mod-np.median(I_mod)) + 1 , label="2500 K")
plt.plot(w_mod, I_star, label="5200 K")
plt.show()

In [ ]:

# RV shift and subtract
#mod_spec =  Spectrum(I_mod, xaxis=w_mod)
#mod_spec2 = Spectrum(I_mod, w_mod)
#star_spec = Spectrum(I_star, w_mod)

def dopplershift(x, RV):
    c = 299792.458
    lambdaShift = x * (RV / c)
    #print("lambdaShift", lambdaShift)
    return x + lambdaShift
    


def dopplershift_and_interp(x, y, RV):
    "Doppler shift and interpolate to original wavelength positions "
    x_prime = dopplershift(x, RV)
    #print(x_prime -x)
    y_prime = np.interp(x, x_prime, y)
    
    return y_prime
    

def RV_differences_spectrum(x,y,RV):
    "Return a the difference in spectrum when one is RV shifted by the RV value given"
    y_prime = dopplershift_and_interp(x, y, RV)
    
    y_diff = y-y_prime
    return y_diff




In [ ]:
# Adding Convolution Into the mix

#import sys
#sys.path.append('../../UsefulModules/')
from IP_Convolution import IPconvolution


In [ ]:
# Convovle to 100K, 50K and 10K
for Res in [10000,50000,100000]:
    print("Convolving to R={} and then RV shifting".format(Res))
    new_w_mod, new_I_mod = IPconvolution(w_mod, I_mod, [min(w_mod),max(w_mod)], R=Res, FWHM_lim=5.0, plot=True, verbose=False)

    for RV in [1]:
        diff_spec = RV_differences_spectrum(new_w_mod, new_I_mod, RV)
        plt.plot(new_w_mod, diff_spec, label="{} km/s".format(RV))
        plt.title("Difference Spectra at R={} ".format(Res))
        plt.legend()
        plt.ylim([-0.02,.02])
    plt.show()



In [ ]:
# RV shift and subtracting and then convolving

for Res in [10000,50000,100000]:
    print("Create a difference spectra then convolve to R={}".format(Res))
    for RV in [1]:
        diff_spec = RV_differences_spectrum(w_mod, I_mod, RV)
        new_w_mod, new_I_mod = IPconvolution(w_mod, diff_spec, [min(w_mod),max(w_mod)], R=Res, FWHM_lim=5.0, plot=False, verbose=False)
        #plt.figure()
        plt.plot(new_w_mod, new_I_mod, label="{} km/s".format(RV))
        plt.title("Convolved Difference Spectra at R={} ".format(Res))
        plt.legend()
    plt.show()


